In [ ]:
# Asyncio Version: async, await


import asyncio
import time
import aiohttp
import threading
import os
import logging
import concurrent.futures

logger = logging.getLogger(__name__)


def setup_download_dir():
    curr_path = os.getcwd()
    output_folder = 'download_files'
    output_path = os.path.join(curr_path, output_folder)
    try:
        output_path = os.path.join(curr_path, output_folder)
        os.mkdir(output_path)
    except FileExistsError as error:
        logger.error(error)
    return output_path


output_path = setup_download_dir()


def download_file(url, session):
    r = None
    output_file = url.split('/')[4]
    try:
        r = session.get(url, allow_redirects=True, verify=True, timeout=5)
    except requests.exceptions.ConnectionError:
        logger.error("Site not rechable", url)
    open(f"{output_path}/{output_file}.png", 'wb').write(r.content)
    return True


async def download_all_files(file_list):
    global threads
    threads = []
    async with aiohttp.ClientSession() as session:
        for url in file_list:
            async with concurrent.futures.ThreadPoolExecutor() as executor:
                executor.map(download_file, url)


if __name__ == '__main__':
    file_list = ["https://unsplash.com/photos/agzJY5jrsAw/download?force=True",
                 "https://unsplash.com/photos/4rDCa5hBlCs/download?force=True",
                 "https://unsplash.com/photos/jFCViYFYcus/download?force=True",
                 "https://unsplash.com/photos/Y8lCoTRgHPE/download?force=True",
                 "https://unsplash.com/photos/4KrQq8Z6Y5c/download?force=True"]

    start_time = time.perf_counter()
    download_all_files(file_list)
    end_time = time.perf_counter()

    exec_time = end_time - start_time

    print(f"No. of threads: {len(threads)}")  # type: ignore
    print(f"Downloaded {len(file_list)} in {exec_time:.2f} seconds")


In [ ]:
# threading Version: using threading module with join()

from asyncio import threads
import requests
import asyncio
import time
import aiohttp
import threading
import os
import logging

logger = logging.getLogger(__name__)


def setup_download_dir():
    curr_path = os.getcwd()
    output_folder = 'download_files'
    output_path = os.path.join(curr_path, output_folder)
    try:
        output_path = os.path.join(curr_path, output_folder)
        os.mkdir(output_path)
    except FileExistsError as error:
        logger.error(error)
    return output_path


output_path = setup_download_dir()


async def download_file(url, session):
    r = None
    output_file = url.split('/')[4]
    try:
        r = session.get(url, allow_redirects=True, verify=True, timeout=5)
    except requests.exceptions.ConnectionError:
        logger.error("Site not rechable", url)
    open(f"{output_path}/{output_file}.png", 'wb').write(r.content)
    return True


async def download_all_files(file_list):
    tasks = []
    async with aiohttp.ClientSession() as session:
        for url in file_list:
            task = threading.Thread(target=download_file, args=(url, session))
            task.start()
            threads.append(task)
        for thread in threads:
            thread.join()
        return threads


if __name__ == '__main__':
    file_list = ["https://unsplash.com/photos/agzJY5jrsAw/download?force=True",
                 "https://unsplash.com/photos/4rDCa5hBlCs/download?force=True",
                 "https://unsplash.com/photos/jFCViYFYcus/download?force=True",
                 "https://unsplash.com/photos/Y8lCoTRgHPE/download?force=True",
                 "https://unsplash.com/photos/4KrQq8Z6Y5c/download?force=True"
                ]

    start_time = time.perf_counter()
    download_all_files(file_list)
    end_time = time.perf_counter()

    exec_time = end_time - start_time

    print(f"No. of threads: {len(threads)}")  # type: ignore
    print(f"Downloaded {len(file_list)} in {exec_time:.2f} seconds")
